In [1]:
import pandas as pd
import numpy as np
import random as rd

# SQL 
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import _mysql

# Natural language processing for sentiment analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# Plot
import pylab as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
output_notebook()
%matplotlib inline

# Sentiment scores converted to a pickle dump file
import pickle

mysqlFilePath = 'mysql://root:@localhost/clientsuccess?charset=utf8&use_unicode=0'

'''
Thie function replaces two special characters that correspond to True and False 
in the MySQL dump file that I received from the startup company that I consulted for.
'''
def replace_special(char):
    try:
        if char.encode('string-escape') == r'\x01':
            return True
        elif char.encode('string-escape') == r'\x00':
            return False
        else:
            return char
    except:
        return char

/Users/kkim/anaconda/envs/ins-env/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Loading BokehJS ...

In [2]:
engine = create_engine(mysqlFilePath, pool_recycle=3600)
connection = engine.connect()

In [4]:
# table_names has all the names of tables that I received as a MySQL dump file.
table_names = pd.read_sql_query("""
    SELECT 
        table_name 
    FROM 
        information_schema.tables 
    WHERE 
        table_schema='clientsuccess';
    """, connection)

In [5]:
# SaaS company ID = tenant_id
# Companies' customer ID = client_id
# start_date = subscription start date
# end_date = subscription end date
tb_client_subscription =  pd.read_sql_query("""
    SELECT 
        tenant_id, client_id, product_id, start_date, end_date, termination_date, amount
    FROM 
        client_subscription;  
        """,connection).applymap(lambda x: replace_special(x))

In [6]:
# Communication records between the SaaS companies and their customers are retrieved. 
# note = email content
tb_client_note = pd.read_sql_query("""
    SELECT 
        id AS client_note_id, note, subject, created_date_time, client_id, interaction_type_id
    FROM
        client_note
    ;
    """, connection)

In [14]:
'''
The most recent date in pdSeriesDates that is before Jan 1 2017 will be returned. 
If nothing found, pd.NaT is returned.
'''
def getmax(pdSeriesDates):
    dates = pdSeriesDates.apply(pd.to_datetime).sort_values(ascending = False)
    for date in dates:
        if pd.Timestamp(date) < pd.Timestamp('2017-01-01'):
            return date
    return pd.NaT

# The most recent date in pdSeriesDates is returned.
def findmax(pdSeriesDates):
    return pdSeriesDates.apply(pd.to_datetime).max()

'''
tb_client is a pandas dataframe that has all the client information including
tenant_id = each client's tenant_id
start_date = very beginning among all the subscriptions
end_date_for_subscription = end date of the most recent subscription
end_date = end date of the most recent subscription that ended before Jan 1 2017.
'''
tb_client = pd.DataFrame()
tb_client['tenant_id'] = tb_client_subscription\
    .groupby('client_id')['tenant_id'].max()
tb_client['start_date'] = tb_client_subscription\
    .groupby('client_id')['start_date'].min()    
tb_client['end_date_for_subscription'] =  tb_client_subscription\
    .groupby('client_id')['end_date'].apply(findmax)
tb_client['end_date'] = tb_client_subscription\
    .groupby('client_id')['end_date']\
    .apply(getmax)
tb_client['client_id'] = tb_client.index

In [39]:
tb_client_new = tb_client[tb_client['end_date'].apply(type) != pd._libs.tslib.NaTType].copy(deep=True)
tb_client_new['churned'] = tb_client_new[['end_date', 'end_date_for_subscription']].\
    apply(lambda x: False if ((x['end_date'] < x['end_date_for_subscription'])\
                              or (type(x['end_date_for_subscription']) == pd._libs.tslib.NaTType)) else True, axis=1)
tb_client = tb_client_new
tb_client_new = None

In [15]:
# Remove all the clients that all the end date is NaT.
mask = tb_client['end_date'].apply(type) == pd._libs.tslib.NaTType
tb_client = tb_client.drop(tb_client[mask].index)

#### Numbers of communications per month for individual clients are computed. 

In [19]:
'''
freq_per_client recieves the id number of a client and returns 
the client's number of communications with its tenant company
per month as a pandas Frame. 
'''
def freq_per_client(client_id, freq):
    g = tb_client_note[tb_client_note['client_id'] == client_id]\
        .groupby(pd.Grouper(key='created_date_time', freq = freq))
    return pd.DataFrame(g.size())

'''
Numbers of communications per month for individual clients are 
stored in the column 'client_note_M_freq' of the tb_client dataframe. 
'''
tb_client['client_note_M_freq'] = tb_client['client_id'].apply(lambda x: freq_per_client(x, 'M'))

'''
fnt_freq_days_to_renewal recieves the id number of a client and retruns
the client's number of communications with its tenant company per month 
in [time_list, freqency_list] format, where time_list is the number of 
days before the last subscription renewal date, and thus it ranges between 
-365 days to 0 day.
'''
def fnt_freq_days_to_renewal(client_id, freq):
    newindex = tb_client.loc[client_id]['client_note_'+freq+'_freq'].index-tb_client.loc[client_id]['end_date']
    temp =  pd.DataFrame(tb_client.loc[client_id]['client_note_'+freq+'_freq'].iloc[:,0].tolist(), index = newindex)
    temp1 = temp.loc[pd.Timedelta(days=-365):pd.Timedelta(days=0)]
    if len(temp1) != 0:
        xx = temp1.index
        xx = xx.days
        yy = temp1.iloc[:,0].tolist()
        return [xx, yy]
    return [None, None]

'''
Numbers of communications per month for individual clients with respect to 
the days before the last subscription renewal are stored in the column 
freq_days_to_renewal.
'''
tb_client['freq_days_to_renewal'] = tb_client['client_id'].apply(lambda x: fnt_freq_days_to_renewal(x, 'M'))

#### Numbers of communications for every 3 months for individual clients are computed.¶

In [20]:
tb_client['client_note_3M_freq'] = tb_client['client_id'].apply(lambda x: freq_per_client(x, '3M'))
tb_client['3M_freq_days_to_renewal'] = tb_client['client_id'].apply(lambda x: fnt_freq_days_to_renewal(x, '3M'))

#### Sentiment scores are computed for every 3 month email notes from individual clients. The NLTK Vader sentiment pyton packageis used. This step takes a while!!! So, the next code block was commented out, but its output was saved in a pickle file.

In [90]:
# '''
# sentiment_scores receives client_id as an input and returns [time_list, sentiment_list], 
# where time_list is the number of days before the last subscription renewal date, and thus
# it ranges between -365 days to 0 day. sentiment_list is the corresponding sentiment scores for 
# every 3 months. 
# '''
# def sentiment_scores(client_id):
#     sid = SentimentIntensityAnalyzer()
#     neutral_dict = 0

#     g = tb_client_note[tb_client_note['client_id'] == client_id]\
#         .groupby(pd.Grouper(key='created_date_time', freq='3M'))\
#         .apply(lambda x: x['note'].str.cat(sep=' '))
#     # remove special characters like markup language angle brackets to speed up the sentiment analysis.
#     g = g.apply(lambda x: re.sub('<[^>]*>', ' ', x))
#     g = g.apply(lambda x: re.sub('[ \t\r\n\v\f]', ' ', x))
#     dict_scores = g.apply(lambda x: sid.polarity_scores(x) if type(x) == str else neutral_dict).tolist()
    
#     # The 'compound' score was used for the sentiment. 
#     scores =[]
#     for i in range(len(dict_scores)):
#         scores.append(dict_scores[i]['compound'])
    
#     newindex = tb_client.loc[client_id]['client_note_3M_freq'].index-tb_client.loc[client_id]['end_date']
#     newindex = [x.days for x in newindex]

#     mask = [True if x >=-365 and x<=0 else False for x in newindex]
#     newindex = [y for y,x in zip(newindex, mask) if x == True]
#     scores = [y for y,x in zip(scores, mask) if x == True]
#     if len(newindex) == 0:
#         newindex = [None]
#     if len(scores) == 0:
#         scores = [None]
#     return [newindex, scores]

# tb_client['sentiment_3M'] = [[[None], [None]]]*len(tb_client)
# sent_list = [[[None], [None]]]*len(tb_client)

# for i in range(len(tb_client)):   
#     sent_list[i] = sentiment_scores(tb_client.iloc[i]['client_id'])
#     print i, " ",

# df_sentiment = pd.DataFrame(sent_list, index = tb_client['client_id'])
# pickle.dump(df_sentiment, open("sentiment.p", "wb"))
# df_sentiment = None

#### Load the pickle file "sentiment.p"

In [32]:
df_sent = pickle.load( open( "sentiment.p", "rb" ) )

#### Total number of communications during 1 year (if data exist) before a renewal date is computed.

In [33]:
def fnt_sum(df):
    return int(df.sum())

tb_client['client_note_total_count'] = tb_client['freq_days_to_renewal']\
    .apply(lambda x: int(sum(x[1])) if x[1] != None else None)

#### Number of email communications for every month is plotted over time. First, only churn cases were considered. As shown in the graph, there was no systematic trend of the communication frequency over time (i.e., no pattern in frequency momentum).

In [42]:
def plot_hist(str_column, isChurned, ymin = 0, ymax = 160):
    mask = tb_client['churned'].tolist()
    if isChurned:
        ma = [x for x, m in zip(range(len(tb_client)), mask) if m == True]
    else:
        ma = [x for x, m in zip(range(len(tb_client)), mask) if m == False]

    p=figure(plot_width=800, plot_height=300, y_range = Range1d(ymin, ymax))

    for i in ma[0:100]:
        [x,y] = tb_client.iloc[i][str_column]
        p.line(x, y, line_width=5, line_alpha = 0.2)
    p.xaxis.axis_label = 'Days to renewal date'
    p.yaxis.axis_label = 'Number of communications per 3 months'
    show(p)
    
plot_hist('freq_days_to_renewal', isChurned = True, ymax = 100)

#### For non-churn cases. Again, there is no systematic frequency momentum. 

In [43]:
plot_hist('freq_days_to_renewal', isChurned=False, ymax = 600)

#### To reduce noise, number of communications for every "3" months is used. Again, I did not see any systematic frequency momentum. 

In [44]:
plot_hist('3M_freq_days_to_renewal', isChurned=True,ymax=300)

In [45]:
plot_hist('3M_freq_days_to_renewal', isChurned=False, ymax = 500)

In [46]:
# end_date = end date of the last subscription
tb_client['end_date'] = tb_client['end_date'].apply(pd.Timestamp)

# start_date = start date of the 1st subscription
tb_client['start_date'] = tb_client['start_date'].apply(pd.Timestamp)
temp = tb_client['end_date'] - tb_client['start_date']

# sub_duration = entire period of subscriptions
tb_client['sub_duration'] = temp.apply(lambda x: x.days)

# last_subscription_start_date = start date of teh last subscription
endDate = tb_client['end_date']
lst = [tb_client_subscription[tb_client_subscription['client_id']==x] for x in tb_client.index]
startdate_list = [lst[i][lst[i]['end_date'].apply(pd.Timestamp) == endDate.iloc[i]]['start_date'].iloc[0]
     for i in range(len(lst))]
tb_client['last_subscription_start_date'] = pd.Series(startdate_list, index = tb_client.index)
temp = tb_client['end_date'].apply(pd.Timestamp) \
    - tb_client['last_subscription_start_date'].apply(pd.Timestamp)

# last_sub_duration = duration of the last subscription. 
tb_client['last_sub_duration'] = temp.apply(lambda x: x.days)

# amount = paid price for subscription
amount_list = [lst[i][lst[i]['end_date'].apply(pd.Timestamp) == endDate.iloc[i]]['amount'].iloc[0]
     for i in range(len(lst))]
tb_client['amount'] = pd.Series(amount_list, index = tb_client.index)

# amount_per_day = paid price for subscription per day on average
tb_client['amount_per_day'] = tb_client['amount'].div(tb_client['last_sub_duration'])

In [48]:
mask_churn = tb_client['churned'].tolist()
mask_non_churn = [not x for x in mask_churn]

p=figure(plot_width=800, plot_height=300)

p.circle(tb_client['client_note_total_count'][mask_non_churn], \
         tb_client['sub_duration'][mask_non_churn], size=10, alpha = 0.4, color='green')
p.circle(tb_client['client_note_total_count'][mask_churn], \
         tb_client['sub_duration'][mask_churn], size=10, alpha = 0.4, color='red')
p.xaxis.axis_label = 'Number of email communications for 1 year'
p.yaxis.axis_label = 'Entire duration of subscriptions'
show(p)

In [50]:
mask_churn = tb_client['churned'].tolist()
mask_non_churn = [not x for x in mask_churn]

p=figure(plot_width=800, plot_height=300, x_range = Range1d(-1, 10000))

p.circle(tb_client['amount_per_day'][mask_non_churn], \
         tb_client['sub_duration'][mask_non_churn], size=10, alpha = 0.4, color='green')
p.circle(tb_client['amount_per_day'][mask_churn], \
         tb_client['sub_duration'][mask_churn], size=10, alpha = 0.4, color='red')
p.xaxis.axis_label = 'Amount paid per day on average'
p.yaxis.axis_label = 'Entire duration of subscriptions'
show(p)

#### The sentiment score over one year period was stored in the column 'sentiment' in the table tb_client. The tb_client dataframe was stored in mysql table, 'kim_all_tenants'.

In [51]:
df_sent = pickle.load( open( "sentiment.p", "rb" ) )
lst =[sum(x) if x!=[None] else None for x in df_sent.loc[:, 1]]
df_sent['sentiment'] = pd.Series(lst, index = df_sent.index)
tb_client['sentiment'] = df_sent['sentiment']

In [275]:
tb_result = tb_client[tb_client['amount_per_day'] != np.inf]
column_names = ['tenant_id', 'start_date', 'end_date_for_subscription', 'end_date',\
               'churned', 'client_note_total_count', 'sub_duration', 'last_subscription_start_date',\
               'last_sub_duration', 'amount_per_day', 'sentiment']
tb_result[column_names].to_sql(con=engine, name='kim_all_tenants', if_exists='replace')